# QR Model Estimation Notebook

This notebook uses the same functions as `estimate.py` for debugging and inspection.

In [ ]:
import sys
sys.path.insert(0, '..')

from pathlib import Path
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

try:
    import scienceplots
    plt.style.use(["science", "no-latex", "grid"])
except ImportError:
    pass

from lobib import DataLoader

# Import functions from estimate.py
from estimate import (
    pl_select,
    load_raw_data,
    compute_median_event_sizes,
    preprocess,
    estimate_event_probabilities,
    estimate_intensities,
    estimate_size_distributions,
    estimate_burst_sizes,
    fit_geometric,
    geometric_pmf,
)

loader = DataLoader()

## Configuration

In [ ]:
TICKER = "T" 

## 1. Load Raw Data

In [ ]:
raw_df = load_raw_data(loader, TICKER)

print(f"Loaded {len(raw_df):,} raw events")
print(f"Date range: {raw_df['date'].min()} to {raw_df['date'].max()}")
print(f"Days: {raw_df['date'].n_unique()}")
print(f"\nColumns: {raw_df.columns}")

## 2. Diagnose Data Issues

Check for unexpected values in `best_bid_nbr` and `best_ask_nbr`.

In [ ]:
# Check range of best_bid_nbr and best_ask_nbr
print("Range of best_bid_nbr and best_ask_nbr:")
print(raw_df.select(
    pl.col("best_bid_nbr").min().alias("min_bid_nbr"),
    pl.col("best_bid_nbr").max().alias("max_bid_nbr"),
    pl.col("best_ask_nbr").min().alias("min_ask_nbr"),
    pl.col("best_ask_nbr").max().alias("max_ask_nbr"),
))

# Check for values outside expected range
outside_range = raw_df.filter(
    pl.col("best_bid_nbr").abs().gt(10) | 
    pl.col("best_ask_nbr").abs().gt(10) |
    pl.col("best_bid_nbr").eq(0) |
    pl.col("best_ask_nbr").eq(0)
)
print(f"\nRows with best_bid_nbr or best_ask_nbr outside [-10,-1] or [1,10]: {len(outside_range):,}")

if len(outside_range) > 0:
    print("\nSample of problematic rows:")
    print(outside_range.select("date", "best_bid_nbr", "best_ask_nbr", "spread", "event").head(20))

In [ ]:
# Distribution of best_bid_nbr and best_ask_nbr
print("Distribution of best_bid_nbr:")
print(raw_df.group_by("best_bid_nbr").len().sort("best_bid_nbr"))

print("\nDistribution of best_ask_nbr:")
print(raw_df.group_by("best_ask_nbr").len().sort("best_ask_nbr"))

## 3. Compute Median Event Sizes

In [ ]:
median_sizes = compute_median_event_sizes(raw_df)

print("Median event sizes by queue level:")
for q in range(1, 5):
    print(f"  Q_{q}: {median_sizes[q]:,.0f}")

## 4. Preprocess Data

In [ ]:
df = preprocess(raw_df, median_sizes)

print(f"Preprocessed {len(df):,} events (filtered from {len(raw_df):,})")
print(f"Removed {len(raw_df) - len(df):,} events ({100*(len(raw_df) - len(df))/len(raw_df):.2f}%)")

In [ ]:
# Check for null imb_bin in preprocessed data
null_imb = df.filter(pl.col("imb_bin").is_null())
print(f"Null imb_bin in preprocessed df: {len(null_imb):,}")

if len(null_imb) > 0:
    print("\nSample of rows with null imb_bin:")
    print(null_imb.head(10))

## 5. Spread Distribution

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# All events
spread_all = raw_df.group_by("spread").len().sort("spread")
spread_all = spread_all.filter(pl.col("spread").le(20))
total = spread_all["len"].sum()
ax1.bar(spread_all["spread"], spread_all["len"] / total, edgecolor="k", alpha=0.7)
ax1.set_xlabel("Spread (ticks)")
ax1.set_ylabel("Proportion")
ax1.set_title("All Events")

# Trades only
trades = raw_df.filter(pl.col("event").is_in(["Trd", "Trd_All"]))
spread_trades = trades.group_by("spread").len().sort("spread")
spread_trades = spread_trades.filter(pl.col("spread").le(20))
total_trades = spread_trades["len"].sum()
ax2.bar(spread_trades["spread"], spread_trades["len"] / total_trades, edgecolor="k", alpha=0.7, color="tab:orange")
ax2.set_xlabel("Spread (ticks)")
ax2.set_ylabel("Proportion")
ax2.set_title("Trades Only")

plt.suptitle(f"{TICKER} - Spread Distributions")
plt.tight_layout()
plt.show()

## 6. Event Probabilities

In [ ]:
event_probs = estimate_event_probabilities(df)
print(f"Event probabilities: {len(event_probs)} rows")
print(event_probs.head(20))

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

for e in ["Add", "Can", "Trd"]:
    data = event_probs.filter(pl.col("event").eq(e) & pl.col("event_q").eq(-1) & pl.col("spread").eq(1)).sort("imb_bin")
    if len(data) > 0:
        ax1.plot(data["imb_bin"], data["proba"], label=e, marker="o", ms=4)

for e in ["Add", "Can"]:
    data = event_probs.filter(pl.col("event").eq(e) & pl.col("event_q").eq(-2) & pl.col("spread").eq(1)).sort("imb_bin")
    if len(data) > 0:
        ax2.plot(data["imb_bin"], data["proba"], label=e, marker="o", ms=4)

for e in ["Create_Ask", "Create_Bid"]:
    data = event_probs.filter(pl.col("event").eq(e) & pl.col("spread").eq(2)).sort("imb_bin")
    if len(data) > 0:
        ax3.plot(data["imb_bin"], data["proba"], label=e, marker="o", ms=4)

ax1.legend(); ax2.legend(); ax3.legend()
ax1.set_title("P(event | spread=1, q=1)")
ax2.set_title("P(event | spread=1, q=2)")
ax3.set_title("P(event | spread>=2)")
plt.tight_layout()
plt.show()

## 7. Intensities

In [ ]:
intensities, dt_data = estimate_intensities(df)
print(f"Intensities: {len(intensities)} rows")
print(intensities)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

spread1 = intensities.filter(pl.col("spread").eq(1)).sort("imb_bin")
spread2 = intensities.filter(pl.col("spread").eq(2)).sort("imb_bin")

ax1.plot(spread1["imb_bin"], spread1["dt"], marker="o", label="spread=1")
ax1.plot(spread2["imb_bin"], spread2["dt"], marker="o", label="spread>=2")
ax1.set_yscale("log")
ax1.set_xlabel("Imbalance")
ax1.set_ylabel("Mean dt (ns)")
ax1.legend()

ax2.hist(np.log10(dt_data["dt"].to_numpy()), bins=80, edgecolor="k", alpha=0.7)
ax2.set_xlabel("log10(dt)")
ax2.set_ylabel("Count")

plt.tight_layout()
plt.show()

## 8. Size Distributions

In [ ]:
size_dist, size_stat = estimate_size_distributions(df, median_sizes)
print(f"Size distributions: {len(size_dist)} rows")
print(size_dist.head(20))

## 9. Burst Sizes

Uses the preprocessed `df` which already has `imb`, `imb_bin`, and `event_q` computed with proper normalization.

In [ ]:
# Check the preprocessed data has correct imbalance
print(f"Q1 median: {median_sizes[1]}")
print(f"\nPreprocessed df has {len(df):,} rows")
print(f"Null imb: {df.filter(pl.col('imb').is_null()).height:,}")
print(f"Null imb_bin: {df.filter(pl.col('imb_bin').is_null()).height:,}")

# Sample of imb_bin distribution
print("\nimb_bin distribution:")
print(df.group_by("imb_bin").len().sort("imb_bin"))

In [ ]:
# Check that out-of-range best_bid_nbr/best_ask_nbr were filtered
# These would have caused null imb_bin in the old code

outside_range_raw = raw_df.filter(
    ~pl.col("best_bid_nbr").is_between(-10, -1) | 
    ~pl.col("best_ask_nbr").is_between(1, 10)
)
print(f"Raw data rows with best_bid_nbr/best_ask_nbr outside expected range: {len(outside_range_raw):,}")

if len(outside_range_raw) > 0:
    print("\nbest_bid_nbr distribution (outside [-10,-1]):")
    out_bid = outside_range_raw.filter(~pl.col("best_bid_nbr").is_between(-10, -1))
    if len(out_bid) > 0:
        print(out_bid.group_by("best_bid_nbr").len().sort("len", descending=True).head(10))
    
    print("\nbest_ask_nbr distribution (outside [1,10]):")
    out_ask = outside_range_raw.filter(~pl.col("best_ask_nbr").is_between(1, 10))
    if len(out_ask) > 0:
        print(out_ask.group_by("best_ask_nbr").len().sort("len", descending=True).head(10))

In [ ]:
# Now run the full burst estimation (uses preprocessed df)
try:
    burst_dist, burst_sizes = estimate_burst_sizes(df, median_sizes)
    print(f"Burst distributions: {len(burst_dist)} rows")
    print(burst_dist)
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Manual step-through of estimate_burst_sizes to inspect the data
# Now uses preprocessed df which already has imb, imb_bin, event_q

q1_median = median_sizes[1]

# Filter to events within Q_±2
df_b = df.filter(pl.col("event_q").abs().le(2))

print(f"Events in Q_±2: {len(df_b):,}")
print(f"Null imb_bin: {df_b.filter(pl.col('imb_bin').is_null()).height:,}")

# Group events into bursts
df_b = df_b.with_columns([
    pl.col("event").is_in(["Create_Bid", "Create_Ask"]).alias("is_create"),
    (pl.col("price") != pl.col("price").shift(1)).alias("price_changed"),
    (~pl.col("event").is_in(["Add", "Create_Bid", "Create_Ask"])).alias("not_add"),
    (pl.col("date") != pl.col("date").shift(1)).alias("new_day"),
])

df_b = df_b.with_columns(
    (pl.col("is_create") | pl.col("price_changed") | pl.col("not_add") | pl.col("new_day"))
    .cum_sum()
    .alias("group_id")
)

burst_sizes_manual = (
    df_b.group_by("group_id")
    .agg([
        pl.col("imb_bin").first(),
        pl.col("event").first(),
        pl.col("is_create").first().alias("starts_with_create"),
        pl.col("event_side").first().alias("side"),
        pl.col("event_size").sum().alias("total_size"),
    ])
    .filter(pl.col("starts_with_create"))
)

print(f"\nBurst sizes: {len(burst_sizes_manual):,}")
print(f"Null imb_bin in bursts: {burst_sizes_manual.filter(pl.col('imb_bin').is_null()).height:,}")

# Show distribution of imb_bin
print("\nimb_bin distribution:")
print(burst_sizes_manual.group_by("imb_bin").len().sort("imb_bin"))